# Hemolymph and hepatopancreas microbiomes of schistosome infected  *Biomphalaria glabrata*


## Aim

We previously characterized the hemolymph (blood) and organ microbiomes of *Biomphalaria* snail species ([Chevalier *et al.*, 2020](https://doi.org/10.1111/1462-2920.15303), [Carruthers *et al.*](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11195231/)) and showed that these microbiomes were diverse. Since these snails are vector of schistosome (blood fluke) parasite, and that this parasite spend a month to develop into the snail, mainly in the hepatopancreas, and are bathed by hemolymph, we investigated the effects of schistosome parasite (blood fluke) infection on the microbiomes of hemolymph and hepatopancreas of its snail vector. We sampled the hemolymph and hepatopancreas of 6-8 snails of infected and uninfected snails each week. These snails were sampled from two replicated trays. We extracted and sequenced the V4 region of the 16S rRNA gene of each sample. We also quantified the number of copy of 16S rRNA gene by qPCR to approximate the bacterial density. In this notebook, we detail the analysis done to characterize the different sample microbiomes and compare them. The results are presented in the manuscript entitled "Limited Impact of Schistosome Infection on *Biomphalaria glabrata* Snail Microbiomes".

Several folders are are already available or will be created during the analysis. Here is the list of the folders and their contents:
* **data**: Sequencing files, qPCR result file, database for taxonomy assignment, sample metadata, and list of contaminants. These files will be either present or downloaded.
* **.env**: Files needed to create appropriate environment.
* **results**: Files generated through data processing. If not existing, this will be created during the analysis.
* **scripts**: Scripts used for the analysis.



## Environment and data

### Creating environment

Creating a conda environment improves reproducibility by installing specific versions of the programs used.

In [ ]:
# Download QIIME2
wget https://data.qiime2.org/distro/core/qiime2-2021.4-py38-linux-conda.yml

In [ ]:
# Check if conda available
[[ ! $(which conda 2> /dev/null) ]] && echo "conda not available in \$PATH. Please interrupt the kernel and fix the situation." && sleep inf

# Creating conda environments
conda env create -n A1qiime2 --file qiime2-2021.4-py38-linux-conda.yml
conda env create -f .env/env.yml

# Cleanup
rm qiime2-2021.4-py38-linux-conda.yml

In [ ]:
# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

In [ ]:
# Activate QIIME2
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate ubiome_bg_inf

In [ ]:
# Install extra R packages
Rscript ".env/R_package_dependencies.R"

Folder and file variables

In [ ]:
# Folders
data="data/libraries"
ldir="$data"
ddir="data/qPCR"
results="results/0 - QIIME2/"
dbdir="data/Silva db"
bldir="data/Blast db"

# Metadata file
md="data/A1_metadata.txt"

### Downloading sequencing data

This step downloads the fastq files of the different samples.

In [ ]:
# Data directory
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Bioproject
bioproject=PRJNA1171869

# Download related information to data project
runinfo=$(esearch -db sra -query ${bioproject} | efetch -format runinfo | sed "/^$/d")

# Field of interest (library name and weblink)
fdn=$(head -n 1 <<< "$runinfo" | tr "," "\n" | grep -w -n "LibraryName" | cut -d ":" -f 1)
fdr=$(head -n 1 <<< "$runinfo" | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    
    # Download
    echo "$fln"
    fastq-dump -O "$ldir" --split-files "$run"
    
    mv "$ldir/${run}_1.fastq" "$ldir/${fln}_R1.fastq"
    mv "$ldir/${run}_2.fastq" "$ldir/${fln}_R2.fastq"
        
done < <(tail -n +2 <<< "$runinfo")

### Downloading databases

The Silva database is used to assign taxonomy to the ASVs generated from the sequencing data.

In [ ]:
# Database directory
[[ ! -d "$dbdir" ]] && mkdir -p "$dbdir"
# creating database directory (Silva db) in data folder
# checking to see if it exists, if not creates it 

# Download and extract the relevant Silva file
wget -P "$dbdir" 'https://www.arb-silva.de/fileadmin/silva_databases/qiime/Silva_132_release.zip'
# -P flag, sets directory prefix to prefix - directory where all other files and subdirectories will be saved to 
# calls database directory and uploads file from link
# 138.1 more recent release, 8/25/2020, but not listed under qiime folder
unzip "$dbdir/Silva_132_release.zip" -d "$dbdir" && rm "$dbdir/Silva_132_release.zip"
# unzips file in directory and removes zipped file
# -d optional directory to which to extract files

# Import the sequence database in Qiime format
qiime tools import \
    --input-path "$dbdir/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna" \
    --output-path "$dbdir/silva_132_99_16S.qza" \
    --type 'FeatureData[Sequence]'
# converts data to QIIME qza format

# Import the taxonomy database in Qiime format
qiime tools import \
    --input-path "$dbdir/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt" \
    --output-path "$dbdir/silva_132_99_16S_taxa.qza" \
    --type 'FeatureData[Taxonomy]' \
    --input-format HeaderlessTSVTaxonomyFormat
# converts data to QIIME qza format

The BLAST nucleotide database is used to identified possible eukaryotic contaminations.

In [ ]:
# Database directory
[[ ! -d "$bldir" ]] && mkdir -p "$bldir"

# Download Blast nucleotide (nt) database
old_pwd="$PWD"
cd "$bldir"
update_blastdb.pl --decompress nt
cd "$old_pwd"

## Qiime pipeline

This section process the data to generate ASVs and assign taxonomy.

In [ ]:
conda deactivate

# Activate QIIME2
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate A1qiime2

In [ ]:
## QIIME PROCRESSING OF MERGED DATA

# Result directory
[[ ! -d "$results" ]] && mkdir -p "$results"

# Check for sequencing data
[[ ! $(find "$data" -type f -name *fastq) ]] && echo  "No sequencing data. Please interrupt the kernel and fix the situation." && sleep inf

# Create the manifest for importing data in artefact
for i in $(ls "$data"/* | sed "s,_R[12].fastq,,g" | uniq) 
do
    nm=$(sed "s,$data/,," <<<$i)
    fl=$(ls -1 "$PWD/$i"* | tr "\n" "\t")

    echo -e "$nm\t$fl"
done > "$results/Manifest"

# Add header
sed -i "1s/^/sample-id\tforward-absolute-filepath\treverse-absolute-filepath\n/" "$results/Manifest"

# Import data
qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path "$results/Manifest" \
    --input-format PairedEndFastqManifestPhred33V2 \
    --output-path "$results/full-demux-single-end.qza"

# Make a summary to check read quality
qiime demux summarize \
    --i-data "$results/full-demux-paired-end.qza" \
    --o-visualization "$results/full-demux-single-end.qzv"

# Run dada2
qiime dada2 denoise-single \
    --i-demultiplexed-seqs "$results/full-demux-single-end.qza" \
    --p-trunc-len 250 \
    --p-max-ee 5 \
    --p-n-threads 0 \
    --o-table "$results/table.qza" \
    --o-representative-sequences "$results/rep-seqs.qza" \
    --o-denoising-stats "$results/denoising-stats.qza"

# Add metadata information to the denoising stats
qiime metadata tabulate \
    --m-input-file "$md" \
    --m-input-file "$results/denoising-stats.qza" \
    --o-visualization "$results/denoising-stats.qzv"


### Taxonomy identification

This step assigns taxonomy to the ASVs generated. This is done using the Silva database and the `classify-consensus-vsearch` method from the `feature-classifier` plugin with an identity threshold of 97%.

In [ ]:
# Assigning taxonomy to ASVs (using Silva database)
qiime feature-classifier classify-consensus-vsearch \
    --i-query "$results/rep-seqs.qza" \
    --i-reference-reads "$dbdir/silva_132_99_16S.qza" \
    --i-reference-taxonomy "$dbdir/silva_132_99_16S_taxa.qza" \
    --p-perc-identity 0.97 \
    --p-threads $(nproc) \
    --o-classification "$results/rep-seqs_taxa.qza"

### Building phylogenetic tree

This step generates a phylogeny from the ASVs ([source](https://chmi-sops.github.io/mydoc_qiime2.html)). ASVs are aligned, masked for highly variable regions. A tree is created from the alignment and rooted in its center.

In [ ]:
# Generates phylogeny from ASVs
[[ -f "results/rooted-tree.qza" ]] && echo "A tree file (rooted-tree.qza) exists already. Please interrupt the kernel and fix the situation." && sleep inf

# Multiple seqeunce alignment using Mafft
qiime alignment mafft \
    --i-sequences "$results/rep-seqs.qza" \
    --o-alignment "$results/aligned-rep-seqs.qza"

# Masking (or filtering) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree.
qiime alignment mask \
    --i-alignment "$results/aligned-rep-seqs.qza" \
    --o-masked-alignment "$results/masked-aligned-rep-seqs.qza"

# Creating tree using the Fasttree program
qiime phylogeny fasttree \
    --i-alignment "$results/masked-aligned-rep-seqs.qza" \
    --o-tree "$results/unrooted-tree.qza"

# Root the tree using the longest root
qiime phylogeny midpoint-root \
    --i-tree "$results/unrooted-tree.qza" \
    --o-rooted-tree "$results/rooted-tree.qza"

## Identification of ASVs with unassigned taxonomy

ASVs with unassigned taxonomy could correspond to eukaryotic contaminants because of the 16S primers amplifying on the 5S or 18S regions. To exclude such contaminants, we perform a megablast search againt the NCBI nt database to find the best sequence similarity to a given unassigned ASV. The results will be then used to exclude non 16S sequences for the subsequent analysis.

Because of the time this analysis can take (up to 7h), the list of contaminants is already available in the data folder.

### Export of unassigned ASV

We use Qiime to filter and export ASV without taxonomy assignments.

In [ ]:
# Filtering assigned ASV out 
qiime taxa filter-seqs \
    --i-sequences "$results/rep-seqs.qza" \
    --i-taxonomy "$results/rep-seqs_taxa.qza" \
    --p-include "Unassigned" \
    --o-filtered-sequences "$results/rep-seqs_unassigned.qza"

# Exporting sequences in fasta format
qiime tools export \
    --input-path "$results/rep-seqs_unassigned.qza" \
    --output-path "$results/rep-seqs-unassigned"

### Blast and annotation of unassigned ASVs

To identify the most similar sequences to the unassigned ASVs, we perform a megablast. This is done using a relatively lenient e-value parameter (1e-2) to increase power of detection. This step is relatively long (estimated running time: 5 h - 8 h) even when using a maxiumum target of 1 with a maximum alignment (HSPs) of 1. The resulting table is then updated with the title and the phyla of the blast match (estimated running time: 30 min - 1 h). Finally we correct the annotations because some ASVs match 16S mitochondrial DNA of eukayote organisms and are wrongly classified as eukaryotes while they likely represent bacteria.

In [ ]:
# Blast the unassigned against the nt database
export BLASTDB="$PWD/$bldir/"
blastn -task megablast -db nt \
    -query "$results/rep-seqs-unassigned/dna-sequences.fasta" \
    -max_target_seqs 1 \
    -max_hsps 1 \
    -evalue 1e-2 \
    -outfmt 6 > "$results/rep-seqs-unassigned/unassigned.blastn.tsv"

# Complete the table to identify what kind of organism the match belongs to
for ((i=1; i <= $(wc -l < "$results/rep-seqs-unassigned/unassigned.blastn.tsv"); i++))
do
    # Get GI from the blast result
    gi=$(sed -n "${i}p" "$results/rep-seqs-unassigned/unassigned.blastn.tsv" | cut -f 2)
    
    # Download entry and get title and phylym info
    entry=$(wget -q -O - "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=${gi}&rettype=gb")
    title=$(echo "$entry" | grep "^DEFINITION" | cut -d " " -f 3-)
    phylum=$(echo "$entry" | grep -A 1 "^ *ORGANISM" | sed -n "2p" | cut -d ";" -f 1 | sed "s/ *//g")
    
    # Update table line
    sed -i "${i}s/$/\t$title\t$phylum/" "$results/rep-seqs-unassigned/unassigned.blastn.tsv"
    
    # Sleep a little to avoid server closing connection on next request
    sleep 0.25s
done

# Correct annotation
for i in $(egrep -n "mitochondri.*Eukaryota$" "$results/rep-seqs-unassigned/unassigned.blastn.tsv" | cut -d ":" -f 1)
do
    sed -i "${i}s/Eukaryota/Bacteria/" "$results/rep-seqs-unassigned/unassigned.blastn.tsv"
done

## Library analysis

To better understand the impact of the different steps on the number of reads filtered out, we analyze the Qiime2 visualization generated after the denoising step. We generate a table summarizing number of reads retained (mean, standard deviation and range) at each step, which includes the number of initial, filtered, denoised, merged and non chimeric reads, for each population and replicate.

In [ ]:
conda deactivate

# Activate ubiome_bg_inf
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate ubiome_bg_inf

In [ ]:
Rscript scripts/Library_Stats.R

## Survival analysis

We analyzed the survival of snails from exposed and control cohorts that have not been sampled for microbiome.

In [ ]:
Rscript scripts/SurvivalAnalysis.R

## Microbiome diversity

We measure several parameters to characterize the different microbiomes:
* **Rarefaction curves**: we generate them for each library to check whether sequencing effort was enough.
* **$\alpha$-diversity**: we measured observed richness, Faith's phylogenetic diversity and Simpson evenness and tested which factor(s) (sample type, infection status, time, etc.) explained the variation observed using linear mixed-effect models.
* **$\beta$-diversity**: we measured Bray-Curtis and weighted UniFrac to understand the dynamic of the microbial communities over time and during infection.
* **Taxonomic diversity**: we investigated which taxa were found in sample types.

Details about the methods used are available in the R scripts. Analysis of the results are detailed in the manuscript.

In [ ]:
Rscript scripts/Create_Phyloseq.R

In [ ]:
Rscript scripts/Rarefaction_Curve.R

In [ ]:
Rscript scripts/Alpha_diversity.R

In [ ]:
Rscript scripts/Beta_Diversity.R

In [ ]:
Rscript scripts/Taxonomy.R

## Microbiome density

We investigate the bacterial density of each hemolymph of infected and uninfected snails over time. Bacterial density was estimated using 16S qPCR. We compare quantities within each snail.

In [ ]:
Rscript scripts/Bacterial_Density.R